In [1]:
import pandas as pd 
import numpy as np
import json 
import pdb 
from tqdm import tqdm

In [2]:
tr_comp = pd.read_csv('./CNN_data/training_dataset/training_compound.csv')
tr_prot = pd.read_csv('./CNN_data/training_dataset/training_protein.csv')
tr_dti = pd.read_csv('./CNN_data/training_dataset/training_dti.csv')


vl_comp = pd.read_csv('./CNN_data/validation_dataset/validation_compound.csv')
vl_prot = pd.read_csv('./CNN_data/validation_dataset/validation_protein.csv')
vl_dti = pd.read_csv('./CNN_data/validation_dataset/validation_dti.csv')

In [3]:
print(tr_comp.shape)
print(tr_prot.shape)
print(tr_dti.shape)
print(vl_comp.shape)
print(vl_prot.shape)
print(vl_dti.shape)

(9586, 3)
(3117, 2)
(17751, 3)
(3704, 3)
(1826, 2)
(4445, 3)


In [4]:
total_comp = pd.concat([tr_comp, vl_comp])
total_prot = pd.concat([tr_prot, vl_prot])
total_dti = pd.concat([tr_dti, vl_dti])

In [5]:
print(total_comp.shape)
print(total_prot.shape)
print(total_dti.shape)

(13290, 3)
(4943, 2)
(22196, 3)


## removing duplicates 

In [6]:
comp_nodupl = total_comp.drop_duplicates()
prot_nodupl = total_prot.drop_duplicates()
dti_nodupl = total_dti.drop_duplicates()

print(comp_nodupl.shape)
print(prot_nodupl.shape)
print(dti_nodupl.shape)

(10347, 3)
(3171, 2)
(22196, 3)


## generating a file to test novel pairs

In [27]:
pid_list = list(dti_nodupl['Protein_ID'])
cid_list = list(dti_nodupl['Compound_ID'])


cid_full_list = list(comp_nodupl['Compound_ID'])
pid_full_list = list(prot_nodupl['Protein_ID'])

In [28]:
pid_list[0]

'P15309'

In [ ]:
from more_itertools import locate 
list(locate(pid_list, lambda x: x == pid_list[0]))

In [51]:
pid_final_test = []
cid_final_test = []


for i in tqdm(cid_full_list):
    for j in pid_full_list:
#         if i in pid_list:
#             ix = list(locate(pid_full_list, lambda x: x == i))
#             cid_temp_list = []
#             for k in ix:
#                 cid_temp_list.append(cid_full_list[k])
            
#             if j in cid_temp_list:
#                 pass
#             else:
        pid_final_test.append(j)
        cid_final_test.append(i)


100%|██████████| 10347/10347 [00:05<00:00, 2067.91it/s]


In [103]:
pid_test_list = []
cid_test_list = []

for i in tqdm(list(set(pid_final_test))):
    if i in pid_full_list:
        ix = list(locate(pid_list, lambda x: x == i))
        cid_temp_list = []
        for k in ix:
            cid_temp_list.append(cid_list[k])

        for j in list(set(cid_final_test)):
            if j not in cid_temp_list:
                pid_test_list.append(i)
                cid_test_list.append(j)

        
        
    

100%|██████████| 3171/3171 [16:16<00:00,  3.25it/s]


In [104]:
len()

32788141

In [105]:
df_file = pd.DataFrame({'Protein_ID': pid_test_list, 'Compound_ID': cid_test_list})
df_file.head()

df_file.to_csv('./test_random_pairs/dti_test.csv')

In [109]:
comp_nodupl.to_csv('./test_random_pairs/dti_comp_noidx.csv', index= False)
prot_nodupl.to_csv('./test_random_pairs/dti_prot_noidx.csv', index = False)

In [110]:
df_file_1 = pd.DataFrame({'Protein_ID': pid_test_list[0:100000], 'CoFalseund_ID': cid_test_list[0:100000]})
df_file_1.head()

df_file_1.to_csv('./test_random_pairs/dti_test_100000_idx.csv', index=False)

## equal split of positive and negative dataset

In [7]:
dti_nodupl.head()

,Protein_ID,Compound_ID,Label
0,P15309,SUB-7688,0
1,Q0ZUL0,SUB-417,1
2,Q13093,SUB-5066,1
3,E5DD06,SUB-7215,0
4,S5M6X9,SUB-1824,1


In [8]:
labels = dti_nodupl['Label'].tolist()

In [9]:
pos_act = sum(labels)
neg_act = len(labels) - pos_act

print('total positive activity pairs: ', pos_act)
print('total negative activity pairs: ', neg_act)

total positive activity pairs:  11100
total negative activity pairs:  11096


In [10]:
positive_df = dti_nodupl[dti_nodupl['Label'] == 1]
negative_df = dti_nodupl[dti_nodupl['Label'] == 0]

In [11]:
print('shape of total positive pairs: ', positive_df.shape)
print('shape of total negative pairs: ', negative_df.shape)

shape of total positive pairs:  (11100, 3)
shape of total negative pairs:  (11096, 3)


# splitting 80-20 dataset

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.utils import shuffle


In [14]:
for i in range(11):
    train_pos, test_pos = train_test_split(positive_df, test_size = 0.1, random_state = i)
    train_neg, test_neg = train_test_split(negative_df, test_size = 0.1, random_state = i)
    
    train_positive, validation_positive = train_test_split(train_pos, test_size = 1/9, random_state = i)
    train_negative, validation_negative = train_test_split(train_neg, test_size = 1/9, random_state = i)
    
    train_concat = pd.concat([train_positive, train_negative])
    test_concat = pd.concat([test_pos, test_neg])
    val_concat = pd.concat([validation_positive, validation_negative])

    train_shuffle = shuffle(train_concat, random_state = i) #.sample(frac=1, random_state = i)
    test_shuffle = shuffle(test_concat, random_state = i) #.sample(frac =1, random_state = i)
    val_shuffle = shuffle(val_concat, random_state = i) #.sample(frac =1, random_state = i)
    
#     print(type(train_shuffle))
#     print(train_shuffle.shape)
    
    train_reindex = train_shuffle.reset_index(drop=True)
    test_reindex = test_shuffle.reset_index(drop=True)
    val_reindex = val_shuffle.reset_index(drop=True)
    
    
    tr_folder = './CNN_data_split/CNN_data_' + str(i) + '/training_dataset/training_act.csv'
    val_folder = './CNN_data_split/CNN_data_' + str(i) + '/validation_dataset/validation_act.csv' 
    test_folder = './CNN_data_split/CNN_data_' + str(i) + '/test_dataset/test_act.csv'
    
    train_reindex.to_csv(tr_folder, index =False)
    val_reindex.to_csv(val_folder, index = False)
    test_reindex.to_csv(test_folder, index = False)
    

In [15]:
print(train_reindex.shape)
print(val_reindex.shape)
print(test_reindex.shape)

(17756, 3)
(2220, 3)
(2220, 3)


In [16]:
11100*0.8

8880.0

In [17]:
11096*0.8

8876.800000000001

In [18]:
8880+8876

17756

In [19]:
for k in tqdm(range(11)):
    train_folder = './CNN_data_split/CNN_data_' + str(k) + '/training_dataset/training_act'
    valid_folder = './CNN_data_split/CNN_data_' + str(k) + '/validation_dataset/validation_act' 
    tes_folder = './CNN_data_split/CNN_data_' + str(k) + '/test_dataset/test_act'
    
    tr_read = pd.read_csv(train_folder)
    val_read = pd.read_csv(valid_folder)
    tes_read = pd.read_csv(tes_folder)

    pid_tr = tr_read['Protein_ID'].to_list()
    pid_val = val_read['Protein_ID'].to_list()
    pid_tes = tes_read['Protein_ID'].to_list()
    
    cid_tr = tr_read['Compound_ID'].to_list()
    cid_val = val_read['Compound_ID'].to_list()
    cid_tes = tes_read['Compound_ID'].to_list()
    
    uniq_pid_tr = list(set(pid_tr))
    uniq_pid_val = list(set(pid_val))
    uniq_pid_tes = list(set(pid_tes))
    
    uniq_cid_tr = list(set(cid_tr))
    uniq_cid_val = list(set(cid_val))
    uniq_cid_tes = list(set(cid_tes))
    
## train    
    for i in range(len(uniq_cid_tr)):
        if i==0:
            train_comp_df = comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_tr[i]]
        else:
            train_comp_df = pd.concat([comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_tr[i]], train_comp_df])
    

    for i in range(len(uniq_pid_tr)):
        if i==0:
            train_prot_df = prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_tr[i]]
        else:
            train_prot_df = pd.concat([prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_tr[i]], train_prot_df])
             
## valid      
    for i in range(len(uniq_cid_val)):
        if i==0:
            valid_comp_df = comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_val[i]]
        else:
            valid_comp_df = pd.concat([comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_val[i]], valid_comp_df])
    

    for i in range(len(uniq_pid_val)):
        if i==0:
            valid_prot_df = prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_val[i]]
        else:
            valid_prot_df = pd.concat([prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_val[i]], valid_prot_df])
            
## test                
    for i in range(len(uniq_cid_tes)):
        if i==0:
            test_comp_df = comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_tes[i]]
        else:
            test_comp_df = pd.concat([comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid_tes[i]], test_comp_df])
    

    for i in range(len(uniq_pid_tes)):
        if i==0:
            test_prot_df = prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_tes[i]]
        else:
            test_prot_df = pd.concat([prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid_tes[i]], test_prot_df])
            

    train_comp_df = train_comp_df.reset_index(drop=True)
    train_prot_df = train_prot_df.reset_index(drop=True)
    
    valid_comp_df = valid_comp_df.reset_index(drop=True)
    valid_prot_df = valid_prot_df.reset_index(drop=True)
    
    test_comp_df = test_comp_df.reset_index(drop=True)
    test_prot_df = test_prot_df.reset_index(drop=True)
    

    
    train_fname_cid = './CNN_data_split/CNN_data_' + str(k) + '/training_dataset/training_compound.csv'
    train_fname_pid = './CNN_data_split/CNN_data_' + str(k) + '/training_dataset/training_protein.csv'
    
    val_fname_cid = './CNN_data_split/CNN_data_' + str(k) + '/validation_dataset/validation_compound.csv' 
    val_fname_pid = './CNN_data_split/CNN_data_' + str(k) + '/validation_dataset/validation_protein.csv' 
    
    test_fname_cid = './CNN_data_split/CNN_data_' + str(k) + '/test_dataset/test_compound.csv'
    test_fname_pid = './CNN_data_split/CNN_data_' + str(k) + '/test_dataset/test_protein.csv'
        
    train_comp_df.to_csv(train_fname_cid, index =False)
    train_prot_df.to_csv(train_fname_pid, index =False)
    
    valid_comp_df.to_csv(val_fname_cid, index = False)
    valid_prot_df.to_csv(val_fname_pid, index =False)
    
    test_comp_df.to_csv(test_fname_cid, index = False)
    test_prot_df.to_csv(test_fname_pid, index =False)
    
    print('#---------------------------------------#')
    
    print('len cid train', len(uniq_cid_tr))
    print('df shape', train_comp_df.shape)

    print('len pid train', len(uniq_pid_tr))
    print('df shape', train_prot_df.shape)

    print('len cid val', len(uniq_cid_val))
    print('df shape', valid_comp_df.shape)

    print('len pid val', len(uniq_pid_val))
    print('df shape', valid_prot_df.shape)
    
    print('len cid test', len(uniq_cid_tes))
    print('df shape', test_comp_df.shape)

    print('len pid test', len(uniq_pid_tes))
    print('df shape', test_prot_df.shape)
    
    print('#---------------------------------------#')

# A = pd.read_csv('./CNN_data_split/CNN_data_0/training_dataset/training_act')

  9%|▉         | 1/11 [00:32<05:22, 32.24s/it]

#---------------------------------------#
len cid train 9607
df shape (9607, 3)
len pid train 3113
df shape (3113, 2)
len cid val 2002
df shape (2002, 3)
len pid val 1237
df shape (1237, 2)
len cid test 2005
df shape (2005, 3)
len pid test 1234
df shape (1234, 2)
#---------------------------------------#


 18%|█▊        | 2/11 [01:03<04:47, 31.95s/it]

#---------------------------------------#
len cid train 9555
df shape (9555, 3)
len pid train 3103
df shape (3103, 2)
len cid val 1994
df shape (1994, 3)
len pid val 1224
df shape (1224, 2)
len cid test 2005
df shape (2005, 3)
len pid test 1249
df shape (1249, 2)
#---------------------------------------#


 27%|██▋       | 3/11 [01:35<04:14, 31.83s/it]

...

#---------------------------------------#
len cid train 9533
df shape (9533, 3)
len pid train 3115
df shape (3115, 2)
len cid val 1980
df shape (1980, 3)
len pid val 1191
df shape (1191, 2)
len cid test 2009
df shape (2009, 3)
len pid test 1231
df shape (1231, 2)
#---------------------------------------#


 91%|█████████ | 10/11 [05:18<00:31, 31.81s/it]

#---------------------------------------#
len cid train 9602
df shape (9602, 3)
len pid train 3119
df shape (3119, 2)
len cid val 2011
df shape (2011, 3)
len pid val 1210
df shape (1210, 2)
len cid test 2010
df shape (2010, 3)
len pid test 1191
df shape (1191, 2)
#---------------------------------------#


100%|██████████| 11/11 [05:50<00:00, 31.89s/it]

#---------------------------------------#
len cid train 9554
df shape (9554, 3)
len pid train 3109
df shape (3109, 2)
len cid val 2012
df shape (2012, 3)
len pid val 1238
df shape (1238, 2)
len cid test 1995
df shape (1995, 3)
len pid test 1253
df shape (1253, 2)
#---------------------------------------#


In [70]:
len(uniq_pid_tes)

1234

In [35]:
pidA = A['Protein_ID'].to_list()
cidA = A['Compound_ID'].to_list()

kjjfkadsjf 3112


In [41]:
uniq_pid = list(set(pidA))
uniq_cid = list(set(cidA))

In [44]:
prot_nodupl.head()

,Protein_ID,Sequence
0,P04335,MTQANLSETLFKPRFKHPETSTLVRRFNHGAQPPVQSALDGKTIPH...
1,R9WYJ0,MSTRKAVIGYYFIPTNQINNYTETDTSVVPFPVSNITPAKAKQLTH...
2,P0A111,MNYNNKILVSESGLSQKHLIHGDEELFQHELKTIFARNWLFLTHDS...
3,P54315,MLIFWTITLFLLGAAKGKEVCYEDLGCFSDTEPWGGTAIRPLKILP...
4,P76270,MNKTEFYADLNRDFNALMAGETSFLATLANTSALLYERLTDINWAG...


In [45]:
comp_nodupl.head()

,Compound_ID,smiles,morgan_fp_r2
0,SUB-6711,C=C,0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0...
1,SUB-9324,C1=CC(=CC=C1[N+](=O)[O-])OC2C(C(C(C(O2)CO)O)O)O,0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0...
2,SUB-6923,C1(C(C(C(C(C1OP(=O)(O)O)OP(=O)(O)O)OP(=O)(O)O)...,0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0...
3,SUB-3175,ClC1=C([C@H](C#N)O)C=CC=C1,0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0...
4,SUB-6690,COC(=O)C=CC1=CC=C(C=C1)O,0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0...


In [58]:
for i in range(len(uniq_cid)):
    if i==0:
        train_comp_df = comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid[i]]
    else:
        train_comp_df = pd.concat([comp_nodupl[comp_nodupl['Compound_ID'] == uniq_cid[i]], train_comp_df])
    
    
for i in range(len(uniq_pid)):
    if i==0:
        train_prot_df = prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid[i]]
    else:
        train_prot_df = pd.concat([prot_nodupl[prot_nodupl['Protein_ID'] == uniq_pid[i]], train_prot_df])
        
        
train_comp_df = train_comp_df.reset_index(drop=True)
train_prot_df = train_prot_df.reset_index(drop=True)
    

,Protein_ID,Sequence
0,P21513,MKRMLINATQQEELRVALVDGQRLYDLDIESPGHEQKKANIYKGKI...
1,P62623,MQILLANPRGFCAGVDRAISIVENALAIYGAPIYVRHEVVHNRYVV...
2,Q552R4,MNKIFGIGNDIVKISRLESSFKRHGDKFLKRAFNEVEISIFKSLNP...
3,A5LGH2,MHGNRTALKIDVLIVGTGPAGAASAALLGTYGVRALVINKYGWTAP...
4,Q9HLD0,MLDDIRGFMNTFSETMFMDVINYSLTRDRYDSVFLQRQNYRDLGQL...


In [60]:
len(uniq_pid)

3113

In [ ]:
train_prot_df

## shell script for folder generation 
